In [18]:
from sqlalchemy import create_engine, Column, Integer, Text, ForeignKey
from sqlalchemy.orm import sessionmaker
# Data injection
engine = create_engine('postgresql://jliu3230@localhost/enshance')
Session = sessionmaker(bind=engine)
session = Session()

# Preprocessing


In [294]:
results = session.execute('''SELECT * from labelling_titledescriptionsubjects order by id''')
# Preprocessing

results = results.fetchall()
subjects = [result[4] for result in results]
doc_ids = [result[0] for result in results]

In [295]:
subjects = list(map(lambda x: x.strip('[').strip(']').strip('u\'').strip('\''), subjects))

In [296]:
import re
subjects_list = list(map(lambda x: re.findall('u\'(.*?)\'', x), subjects))

In [302]:
ex14001 = subjects[14000]

In [304]:
ex14001 = clean_doc(ex14001)

In [305]:
vectorizer2 = CountVectorizer(min_df=1)
vectorizer2.fit_transform([ex14001])

<1x4 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [322]:
ex_vector = np.asarray([np.asarray(model[x]) for x in vectorizer2.get_feature_names()])

In [325]:
ex_vector.shape

(4, 300)

In [327]:
len(ex_vector)

4

In [376]:
with open("exvector_noscience", 'wb') as f:
    np.save(f, np.delete(ex_vector, (1), 0))

In [ ]:
doc_ids

In [23]:
# get taxonomy terms
engine2 = create_engine('postgresql://jliu3230@localhost/sharetaxonomy')
Session2 = sessionmaker(bind=engine2)
session2 = Session2()
results2 = session2.execute('''SELECT term from terms''')
terms = [result for result in results2]

In [17]:
# Word2Vec
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [32]:
example = subjects_list[10]


In [33]:
sentences = [subject.split(' ') for subject in example]

In [196]:
sentences

[['coding', 'theory']]

In [5]:
model = gensim.models.Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [25]:
terms[0]

('293T cells',)

In [48]:
# Keep training
total = 0
missing = []
for tv in terms:
    for t in tv[0].split(" "):
        total += 1
        try: 
            model[t]
        except:
            missing.append(tv)
            break
print(total)
print(missing)

18383
[('293T cells',), ('ADP-ribosylation',), ('Abaxial surface',), ('Abscisic acid',), ('Absorptiometry',), ('Acanthodii',), ('Access to consciousness',), ('Acetates',), ('Acetic acid',), ('Acetones',), ('Acetonitrile',), ('Acetylation',), ('Achondroplasia',), ('Acid-fast stain',), ('Acinar morphogenesis',), ('Acremonium',), ('Acridines',), ('Acrosomes',), ('Actinium',), ('Actinobacteria',), ('Actinomadura',), ('Actinomyces',), ('Actinopterygii',), ('Actins',), ('Acute lymphoblastic leukemia',), ('Acylation',), ('Adamantanes',), ('Adaxial surface',), ('Adeno-associated viruses',), ('Adenocarcinoma of the lung',), ('Adenocarcinomas',), ('Adenoidectomy',), ('Adenylyl cyclase',), ('Adenylyl cyclase signaling cascade',), ('Adhesins',), ('Adipocyte differentiation',), ('Adipocytes',), ('Adipokines',), ('Adjustment of dosage at steady state',), ('Adrenalectomy',), ('Adrenarche',), ('Adrenocorticotropic hormone',), ('Advection',), ('Aequorin',), ('Aeromonas hydrophila',), ('Aestivation',), 

Total words: 20140; missing words: 3003

In [120]:
model.similarity('mathematics', 'biological')

0.16314940535165884

In [144]:
from numpy.linalg import norm
norm(model['computer'] - model['physical'])+norm(model['science'] - model['science'])

KeyError: '15'

- Remove stop words from terms
- Remove punctuations
- Split subjects by ", " and " - "


In [ ]:
# Remove stop words from taxonomy terms

# get stop words
import requests
import bs4

html = requests.get("http://www.ranks.nl/stopwords").content
soup = bs4.BeautifulSoup(html)

In [ ]:
soup

In [509]:
str(soup.find_all('td')[0:2])

'[<td valign="top">a<br/>about<br/>above<br/>after<br/>again<br/>against<br/>all<br/>am<br/>an<br/>and<br/>any<br/>are<br/>aren\'t<br/>as<br/>at<br/>be<br/>because<br/>been<br/>before<br/>being<br/>below<br/>between<br/>both<br/>but<br/>by<br/>can\'t<br/>cannot<br/>could<br/>couldn\'t<br/>did<br/>didn\'t<br/>do<br/>does<br/>doesn\'t<br/>doing<br/>don\'t<br/>down<br/>during<br/>each<br/>few<br/>for<br/>from<br/>further<br/>had<br/>hadn\'t<br/>has<br/>hasn\'t<br/>have<br/>haven\'t<br/>having<br/>he<br/>he\'d<br/>he\'ll<br/>he\'s<br/>her<br/>here<br/>here\'s<br/>hers<br/>herself<br/>him<br/>himself<br/>his<br/>how<br/>how\'s<br/>i<br/>i\'d<br/>i\'ll<br/>i\'m<br/>i\'ve<br/>if<br/>in<br/>into<br/>is<br/>isn\'t<br/>it<br/>it\'s<br/>its<br/>itself<br/>let\'s<br/>me<br/>more<br/>most<br/>mustn\'t<br/>my<br/>myself<br/>no<br/>nor<br/>not<br/>of<br/>off<br/>on<br/>once<br/>only<br/>or<br/>other<br/>ought<br/>our<br/>ours</td>, <td valign="top">ourselves<br/>out<br/>over<br/>own<br/>same<br/>shan

In [510]:
stopwords = str(soup.find_all('td')[0])[17:-5].split("<br/>") + str(soup.find_all('td')[1])[17:-5].split("<br/>")

In [569]:
# get the collections of terms for each root
# Computer science
collections = {}
roots = (991, 2092, 2911, 2948, 3208, 5930, 7398, 7626, 8600, 8912, 9268)
for root in roots:
    tree = session2.execute('select * from taxonomytree where array_length(path, 1) < 10 and path[1] = {};'.format(root)).fetchall()
    child_terms = [row[3] for row in tree]
    child_ids = [row[2] for row in tree]
    collections[root] = [(tree[0][0], tree[0][1])] + [x for x in zip(child_ids, child_terms)]


In [570]:
collections[2092]

[(2092, 'Computer and information sciences'),
 (602, 'Artificial intelligence'),
 (1483, 'Catalogs'),
 (2094, 'Computer applications'),
 (2095, 'Computer architecture'),
 (2105, 'Computer modeling'),
 (2106, 'Computer networks'),
 (2107, 'Computer peripherals'),
 (2108, 'Computer security'),
 (2109, 'Computer software'),
 (2110, 'Computer systems analysis'),
 (2112, 'Computer vision'),
 (2117, 'Computerized simulations'),
 (2118, 'Computers'),
 (2119, 'Computing methods'),
 (2120, 'Computing systems'),
 (2342, 'Cryptography'),
 (2520, 'Data acquisition'),
 (2521, 'Data compression'),
 (2522, 'Data management'),
 (2526, 'Data visualization'),
 (4048, 'Geoinformatics'),
 (4203, 'Graph theory'),
 (4759, 'Hypertext'),
 (4953, 'Information technology'),
 (4954, 'Information theory'),
 (5463, 'Library science'),
 (6594, 'Network analysis'),
 (6600, 'Neural networks'),
 (6994, 'Operating systems'),
 (8066, 'Programming languages'),
 (8444, 'Random number generators'),
 (8484, 'Real time compu

In [571]:
# join all terms for each collection
collection_docs = {}
for key in collections.keys():
    collection_docs[key] = " ".join([termtuple[1] for termtuple in collections[key]])

In [572]:
collection_docs[2092]

'Computer and information sciences Artificial intelligence Catalogs Computer applications Computer architecture Computer modeling Computer networks Computer peripherals Computer security Computer software Computer systems analysis Computer vision Computerized simulations Computers Computing methods Computing systems Cryptography Data acquisition Data compression Data management Data visualization Geoinformatics Graph theory Hypertext Information technology Information theory Library science Network analysis Neural networks Operating systems Programming languages Random number generators Real time computing Software engineering Systems science Artificial neural networks Machine learning Genetic programming Web-based applications Computer-aided design Computer-aided manufacturing Computer-assisted instruction Computer hardware User interfaces Pipelines (computing) Bandwidth (computing) Internet Computer keyboards Open source software Systems software Target detection Personal computers M

In [573]:
# build corpus
collection_corpus = list(collection_docs.values())
collection_corpus = list(map(clean_doc, collection_corpus))

In [574]:
collection_corpus[8]

'computer information sciences artificial intelligence catalogs computer applications computer architecture computer modeling computer networks computer peripherals computer security computer software computer systems analysis computer vision computerized simulations computers computing methods computing systems cryptography data acquisition data compression data management data visualization geoinformatics graph theory hypertext information technology information theory library science network analysis neural networks operating systems programming languages random number generators real time computing software engineering systems science artificial neural networks machine learning genetic programming web based applications computer aided design computer aided manufacturing computer assisted instruction computer hardware user interfaces pipelines computing bandwidth computing internet computer keyboards open source software systems software target detection personal computers mainframe

In [575]:
# analyze docs using scikit learn
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)
analyze = vectorizer.build_analyzer()
analyze(collection_corpus[1])

['engineering',
 'technology',
 'aerospace',
 'engineering',
 'architectural',
 'engineering',
 'automotive',
 'engineering',
 'bioengineering',
 'chemical',
 'engineering',
 'civil',
 'engineering',
 'control',
 'engineering',
 'digital',
 'imaging',
 'earthquake',
 'engineering',
 'electrical',
 'engineering',
 'electronics',
 'electronics',
 'engineering',
 'energy',
 'power',
 'environmental',
 'engineering',
 'equipment',
 'fire',
 'engineering',
 'fluidics',
 'forensic',
 'engineering',
 'geological',
 'engineering',
 'human',
 'factors',
 'engineering',
 'industrial',
 'engineering',
 'instrumentation',
 'management',
 'engineering',
 'manufacturing',
 'processes',
 'marine',
 'engineering',
 'measurement',
 'mechanical',
 'engineering',
 'membrane',
 'technology',
 'military',
 'engineering',
 'mining',
 'engineering',
 'nanotechnology',
 'navigation',
 'nuclear',
 'engineering',
 'production',
 'engineering',
 'reliability',
 'engineering',
 'remote',
 'sensing',
 'sanitary',


In [576]:
X = vectorizer.fit_transform(collection_corpus)

In [577]:
vectorizer.get_feature_names()

['ab',
 'abdomen',
 'abdominal',
 'aberrations',
 'abiotic',
 'ablation',
 'abnormal',
 'abnormalities',
 'abortion',
 'abscess',
 'abscesses',
 'absence',
 'absorption',
 'abstract',
 'abundance',
 'abuse',
 'acacia',
 'academic',
 'acanthamoeba',
 'acceleration',
 'accelerators',
 'accelerometers',
 'acceptors',
 'access',
 'accretion',
 'accumbens',
 'ace',
 'acetaldehyde',
 'acetate',
 'acetates',
 'acetic',
 'acetone',
 'acetonitrile',
 'acetylation',
 'acetylcholine',
 'acetylene',
 'achievement',
 'achondroplasia',
 'acid',
 'acidic',
 'acidification',
 'acidosis',
 'acids',
 'acinar',
 'acinetobacter',
 'acne',
 'acorn',
 'acoustic',
 'acoustics',
 'acquired',
 'acquisition',
 'acromegaly',
 'acrylics',
 'actin',
 'actinic',
 'action',
 'activated',
 'activation',
 'activators',
 'active',
 'activity',
 'actuators',
 'acuity',
 'acuminata',
 'acupuncture',
 'acute',
 'acyclic',
 'adamantanes',
 'adaptation',
 'adaptive',
 'addiction',
 'addicts',
 'addition',
 'additive',
 'ade

In [578]:
import numpy as np

In [596]:
num_of_words = np.sum(X.toarray(), axis=1)

In [598]:
num_of_words

array([   70,   760,   328,   569,  3545,   915,  7945,   660,   362,
        3013, 11404], dtype=int64)

In [600]:
collection_weights.shape

(11, 6886)

In [599]:
collection_weights = X.toarray() / num_of_words[:, None]

In [582]:
# Make sure weights add up to 1
np.sum(collection_weights, axis=1)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [583]:
collection_vectors = [model[x] for x in vectorizer.get_feature_names()]

In [584]:
collection_vectors[0]

array([  2.14512441e-02,   7.96760544e-02,   1.79835614e-02,
         6.32247180e-02,  -2.49995701e-02,   6.58053234e-02,
        -1.70158371e-02,  -5.38700446e-02,  -8.03212002e-02,
         3.09672095e-02,  -2.88704727e-02,  -6.61278963e-02,
        -8.09663534e-02,   7.67728761e-02,  -2.32254080e-02,
         1.08385235e-01,  -3.51606868e-02,   7.22568259e-02,
         1.19352788e-02,   5.90312444e-02,  -3.22575122e-02,
         1.32255796e-02,  -2.05641631e-02,   6.90310746e-02,
        -1.01933733e-01,  -4.19347622e-02,  -1.07094936e-01,
         9.63693112e-03,   1.20965661e-02,  -5.72570832e-03,
         6.61278963e-02,   6.90310746e-02,   3.11284978e-02,
        -3.93541642e-02,  -1.24513991e-01,  -1.01933733e-01,
        -7.19342455e-02,   1.01933733e-01,   1.17417336e-01,
        -2.62092275e-04,  -1.37094418e-02,  -3.90315875e-02,
        -3.02414154e-03,   2.03222316e-02,   1.17739914e-02,
         2.75801718e-02,  -1.83968616e-04,  -4.83862646e-02,
        -3.14510725e-02,

In [585]:
# serialize
import pickle

In [312]:
with open("collection_vectors.pickle", 'wb') as f:
    np.save(f, collection_vectors)
with open("collection_weights.pickle", 'wb') as f:
    np.save(f, collection_weights)

In [260]:
# Now deal with subjects from the data
subjects_list

[['simulation', 'msimd', 'mmb'],
 [],
 ['cancer', 'tumor progression', 'hallmarks of cancer'],
 ['superior colliculus', 'delays'],
 ['loreta'],
 [],
 [],
 ['mix networks', 'pseudonyms', 'privacy enhancing techniques'],
 [],
 ['fat tails',
  'l\\xe9vy process',
  'generalized normal-laplace distribution',
  'brownian-laplace motion',
  'financial returns'],
 ['coding theory'],
 ['secondary 62f35', 'primary 62k05', '62m10'],
 ['ie50', 'ie20'],
 ['destructive duration testing',
  'nomial response',
  'nonparametric',
  'latent variable models. 2'],
 ['dynamic auctions', 'terms economics'],
 [],
 [],
 ['social interaction', 'stochastic'],
 ['dynamical systems', 'bucket brigade', 'assembly line'],
 [],
 ['likelihood inference'],
 ['tensor dissimilarity metric',
  'diffusion tensor magnetic resonance imaging',
  'j-divergence',
  'segmentation'],
 ['smart artifacts',
  'emotional awareness',
  'multi-user communication',
  'music',
  'tangible user interfaces'],
 [],
 ['minimal perfect hash'

In [266]:
# First just try a few example
subjects

[]

In [333]:
len(collection_vectors)

1912

In [495]:
collection_docs.values()

dict_values(['Science policy Bioethics Material transfer agreements Open science Research funding Research integrity Science and technology workforce Science education Science policy and economics Technology regulations Sanctity of life Justice in science Scientific beneficence Scientific nonmaleficence Respect for human dignity Open source software Open data Open access Institutional funding of science Military funding of science Philanthropic funding of science Research grants Corporate funding of science Government funding of science Scientific misconduct Publication ethics Careers in research Citizen science Science fairs', 'Engineering and technology Aerospace engineering Architectural engineering Automotive engineering Bioengineering Chemical engineering Civil engineering Control engineering Digital imaging Earthquake engineering Electrical engineering Electronics Electronics engineering Energy and power Environmental engineering Equipment Fire engineering Fluidics Forensic engin

In [360]:
# see if the word science is dominant

vectorizer.get_feature_names()[1588]

'science'

In [361]:
collection_weights[:,1588]

array([ 0.1969697 ,  0.        ,  0.02380952,  0.        ,  0.        ,
        0.00384615,  0.00072516,  0.00584795,  0.00692042,  0.01492537,
        0.00823045])

In [373]:
with open("collection_vectors_noscience", 'wb') as f:
    np.save(f, np.delete(collection_vectors, (1588), 0))
with open("collection_weights_noscience", 'wb') as f:
    np.save(f, np.delete(collection_weights, 1588, 1))

In [372]:
np.delete(collection_vectors, (1588),0)

array([[-0.00233998,  0.05383663, -0.01899312, ..., -0.03962595,
         0.00915496,  0.08417097],
       [ 0.02503812,  0.08263947,  0.00800399, ...,  0.04597163,
        -0.01238224, -0.00957742],
       [ 0.01973061,  0.04352341, -0.02669436, ..., -0.04323325,
        -0.02350264, -0.01341972],
       ..., 
       [ 0.04056438, -0.00354571,  0.02910054, ..., -0.08818345,
         0.03277485,  0.01212522],
       [-0.0032539 ,  0.07331233, -0.07809357, ...,  0.00660741,
         0.11209349,  0.08234356],
       [ 0.05163179, -0.08125658,  0.03442119, ...,  0.01833916,
         0.08859225,  0.0634817 ]], dtype=float32)

In [ ]:
collection_vectors

In [421]:
doc_dist(ex_vector, np.asarray(collection_vectors)[np.where(collection_weights[0]>0)])


4.4900044798851013

In [422]:
doc_dist(ex_vector, np.asarray(collection_vectors)[np.where(collection_weights[8]>0)])


4.4746392369270325

In [423]:
doc_dist(ex_vector, np.asarray(collection_vectors)[np.where(collection_weights[9]>0)])

4.4900044798851013

In [434]:
for i in range(11):
    print(doc_dist(np.delete(ex_vector,(1),0), np.asarray(collection_vectors)[np.where(collection_weights[i]>0)]))

3.16818070412
3.1491856575
3.17494273186
3.21287029982
3.10120129585
3.14773309231
3.1464022398
3.1501879096
3.15641778708
3.16438943148
3.12836682796


In [384]:
np.repeat(ex_vector, len(collection_vectors), 0)

array([[ 0.04081374, -0.07643302,  0.04675029, ..., -0.01827343,
        -0.09424265, -0.02356066],
       [ 0.04081374, -0.07643302,  0.04675029, ..., -0.01827343,
        -0.09424265, -0.02356066],
       [ 0.04081374, -0.07643302,  0.04675029, ..., -0.01827343,
        -0.09424265, -0.02356066],
       ..., 
       [ 0.00726699,  0.07126336,  0.01687816, ..., -0.05738576,
        -0.03338126, -0.01584672],
       [ 0.00726699,  0.07126336,  0.01687816, ..., -0.05738576,
        -0.03338126, -0.01584672],
       [ 0.00726699,  0.07126336,  0.01687816, ..., -0.05738576,
        -0.03338126, -0.01584672]], dtype=float32)

In [417]:
word_dist(ex_vector, np.asarray(collection_vectors)[np.where(collection_weights[0]>0)])

ValueError: operands could not be broadcast together with shapes (4,300) (39,300) 

In [399]:
list(collection_weights[0]>0)

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 F

In [416]:
np.asarray(collection_vectors)[np.where(collection_weights[0]>0)]

array([[ -3.24109979e-02,  -6.91675991e-02,  -5.61307780e-02, ...,
         -6.55462593e-02,  -3.82051431e-02,  -5.86657152e-02],
       [ -5.84943779e-02,   6.13096170e-02,   6.31864369e-02, ...,
         -4.44181897e-02,  -6.22480288e-02,  -3.98825295e-02],
       [  7.58023337e-02,  -5.59570901e-02,  -2.92798728e-02, ...,
         -2.63518840e-02,  -9.21350147e-05,   1.07359532e-02],
       ..., 
       [  1.91709716e-02,   8.07765638e-04,   8.14227760e-02, ...,
          5.33125317e-03,   7.96995405e-03,   9.13313702e-02],
       [  2.32083932e-03,   8.95680711e-02,  -1.70739125e-02, ...,
         -6.41904473e-02,  -5.97120449e-02,   6.26976490e-02],
       [ -6.32887706e-02,  -6.44772947e-02,  -5.38548827e-03, ...,
         -2.02048663e-02,   1.84220839e-02,  -4.93236445e-02]], dtype=float32)

In [429]:
collection_weights.shape

(11, 1912)

In [436]:
### try title and description

titles = [result[1] for result in results]
descriptions = [result[2] for result in results]

In [440]:
td14001 = clean_doc(clean_doc(titles[14000] + descriptions[14000]))
td14001

'code injection attacks html based mobile apps html based mobile apps become popular mostly they much easier ported across different mobile platforms than native apps html based apps implemented using the standard web technologies including html javascript css they depend some such interact with the underlying os knowing that javascript subject code injection attacks we conducted systematic study html based mobile apps trying evaluate whether safe rely the web technologies mobile app development discoveries quite surprising we found out that html based mobile apps become popular seems go that direction based the current projection many the things that we normally today may become dangerous including reading d barcodes scanning wi fi access points playing mp videos pairing with bluetooth devices etc this paper describes html based apps can become vulnerable attackers can exploit their vulnerabilities through variety channels what damage can achieved the attackers addition demonstrating 

In [443]:
vectorizer3 = CountVectorizer(min_df=1, stop_words=stopwords)
vectorizer3.fit_transform([td14001])

<1x105 sparse matrix of type '<class 'numpy.int64'>'
	with 105 stored elements in Compressed Sparse Row format>

In [445]:
td_vector = np.asarray([np.asarray(model[x]) for x in vectorizer3.get_feature_names()])

In [446]:
for i in range(11):
    print(doc_dist(td_vector, np.asarray(collection_vectors)[np.where(collection_weights[i]>0)]))

134.433666766
133.056583166
135.546785116
137.429569542
131.584152818
133.831338227
133.211488664
134.446909785
133.169946253
134.797609091
132.813707352


In [447]:
title_sample = titles[14002:14021]
description_sample = descriptions[14002:14021]
td_sample = list(map(lambda x: " ".join(x), zip(title_sample, description_sample)))

In [449]:
td_sample = list(map(clean_doc, td_sample))

In [450]:
td_sample[1]

'tolerances induced coverings this paper we consider tolerances induced coverings tolerance r u determines r setting x r y r x r y we prove that tolerance r induced covering h u the covering h the set u r bounded minimal elements the tolerance r coincides with the product r circ r we also show that such case h m mid text m minimal u r minimal m we r m m additionally this covering h inducing r consists some blocks the tolerance r we give necessary sufficient conditions under which h the set r blocks coincide these results established applying the notion numbers sets'

In [469]:
vectorizer3 = CountVectorizer(min_df=1, stop_words=stopwords)
X3 = vectorizer3.fit_transform(td_sample)
min_dist = []
for i in range(19):
    td_sample_vector = np.asarray([np.asarray(model[x]) for x in np.asarray(vectorizer3.get_feature_names())[np.where(X3.toarray()[i,:]>0)]])
    distances = [doc_dist(td_sample_vector, np.asarray(collection_vectors)[np.where(collection_weights[j]>0)]) for j in range(11)]
    val, idx = min((val, idx) for (idx, val) in enumerate(distances))
    min_dist += (val,idx)

In [470]:
print(min_dist)

[85.966861248016357, 4, 60.555157840251923, 8, 57.169678807258606, 8, 90.449615836143494, 8, 49.705346584320068, 8, 94.000181972980499, 4, 79.008847713470459, 4, 44.072072625160217, 4, 110.88178604841232, 8, 72.868848085403442, 4, 103.62107187509537, 4, 85.21269029378891, 4, 50.737401187419891, 4, 54.642481684684753, 8, 61.286561667919159, 4, 42.036845326423645, 4, 62.107015907764435, 4, 131.30007416009903, 4, 80.933961391448975, 4]


In [464]:
np.asarray(vectorizer3.get_feature_names())[np.where(X3[1,:]>0)]

array(['additionally', 'also', 'applying', 'blocks', 'bounded', 'case',
       'circ', 'coincide', 'coincides', 'conditions', 'consider',
       'consists', 'covering', 'coverings', 'determines', 'elements',
       'established', 'give', 'induced', 'inducing', 'mid', 'minimal',
       'necessary', 'notion', 'numbers', 'paper', 'product', 'prove',
       'results', 'set', 'sets', 'setting', 'show', 'some', 'such',
       'sufficient', 'text', 'that', 'the', 'these', 'this', 'tolerance',
       'tolerances', 'under', 'we', 'which', 'with'], 
      dtype='<U17')

In [ ]:
X3.toarray()[1,:]

In [477]:
# Try basic word count method
sample_vocab = np.asarray(vectorizer3.get_feature_names())
collection_vocab = np.asarray(vectorizer.get_feature_names())
sample_X = X3.toarray()
collection_X = X.toarray()


In [494]:
for sample_dist in sample_X:
    sample_words = list(sample_vocab[np.where(sample_dist>0)])
    print(sample_words)
    count = []
    min_dist = []
    for collection_dist in collection_X:
        collection_words = list(collection_vocab[np.where(collection_dist>0)])
        # print(len([x for x in sample_words if x in collection_words]))
        print(len([x for x in sample_words if x in collection_words]))
        print([x for x in sample_words if x in collection_words])
    #min_dist += (min((val, idx) for (idx, val) in enumerate(count)))

['also', 'appear', 'around', 'bar', 'bound', 'breaking', 'calculated', 'can', 'channels', 'charged', 'comes', 'consistent', 'consists', 'coupling', 'decay', 'dependent', 'difference', 'energy', 'experiments', 'explain', 'fit', 'formation', 'found', 'give', 'gives', 'hybrid', 'information', 'interaction', 'introduced', 'investigated', 'lack', 'large', 'mass', 'meson', 'mesons', 'mixing', 'model', 'narrow', 'neutral', 'obtained', 'omega', 'origin', 'parameter', 'peak', 'peaks', 'pi', 'present', 'processes', 'psi', 'quark', 'ratios', 'rho', 'size', 'spectrum', 'state', 'strengths', 'sufficiently', 'symmetry', 'taken', 'that', 'the', 'threshold', 'times', 'transfer', 'two', 'up', 'values', 'very', 'we', 'which', 'widths', 'with']
1
['transfer']
5
['energy', 'mass', 'peak', 'processes', 'values']
0
[]
0
[]
10
['energy', 'information', 'interaction', 'mass', 'model', 'processes', 'size', 'spectrum', 'transfer', 'two']
3
['decay', 'information', 'state']
3
['information', 'processes', 'the']


In [ ]:
print(min_dist)

In [ ]:
# Prepare data: subjects, title and description, title description subjects

**Test functions here**

In [518]:
list((map(clean_doc, td_sample)))

['origin narrow peak symmetry breaking x x formation decay processes b decay investigated c bar c two meson hybrid model two meson state consists d bar d d d j psi rho j psi omega channels energy dependent decay widths rho omega mesons introduced d bar d interaction taken consistent lack b bar b bound state coupling d bar d j psi rho d bar d j psi omega channels obtained quark model c bar c d bar d coupling taken parameter fit x mass spectrum calculated found narrow j psi rho j psi omega peaks appear around d bar d threshold size j psi pi peak calculated times large j psi pi symmetry breaking present model comes mass difference charged neutral d d mesons gives sufficiently large mixing explain experiments also found values ratios transfer strengths can give information x mass size c bar c d bar d coupling',
 'tolerances induced coverings paper consider tolerances induced coverings tolerance r u determines r setting x r y r x r y prove tolerance r induced covering h u covering h set u r

In [609]:
label_docs(td_sample, vectorizer)

(6886,)


[(20, 9),
 (4, 6),
 (13, 9),
 (18, 9),
 (10, 9),
 (15, 10),
 (20, 4),
 (9, 9),
 (22, 9),
 (13, 10),
 (23, 10),
 (17, 10),
 (11, 9),
 (9, 4),
 (15, 6),
 (10, 6),
 (8, 6),
 (27, 4),
 (14, 9)]

In [588]:
label_docs(td_sample, vectorizer, method='wmd')

[77.282143354415894,
 9,
 47.915342271327972,
 6,
 50.157800137996674,
 6,
 79.739393889904022,
 8,
 43.566191673278809,
 8,
 85.53216427564621,
 9,
 70.112110674381256,
 9,
 39.243817508220673,
 9,
 99.359461903572083,
 9,
 67.603513300418854,
 9,
 95.707457363605499,
 9,
 75.504769504070282,
 9,
 44.473854541778564,
 9,
 49.384880602359772,
 8,
 52.977329611778259,
 9,
 37.398934066295624,
 9,
 55.180808842182159,
 9,
 122.38315618038177,
 9,
 72.874050676822662,
 9]

In [608]:
label_docs(td_sample, vectorizer, method='count_weight')

(6886,)


[(0.035180882841022242, 9),
 (0.0055248618784530384, 8),
 (0.017922336541652838, 9),
 (0.044287729196050778, 4),
 (0.017489421720733429, 4),
 (0.030386740331491711, 8),
 (0.044569816643159364, 4),
 (0.015300546448087432, 5),
 (0.045133991537376586, 4),
 (0.03591160220994475, 8),
 (0.29999999999999999, 0),
 (0.040902679830747524, 4),
 (0.066298342541436461, 8),
 (0.033149171270718231, 8),
 (0.0086292731496847002, 9),
 (0.0082973780285429816, 9),
 (0.01692665117822768, 9),
 (0.18571428571428572, 0),
 (0.02389644872220378, 9)]

In [623]:
# Prepare documents
# read labels
truelabelscsv = np.genfromtxt ('truelabels.csv', delimiter=",").astype(int)


In [677]:
all_labels = list(enumerate(collection_docs.keys()))

In [627]:
# subjects only, title_description
# sample ids
sample_ids = truelabelscsv[:,0]
truelabels = truelabelscsv[:,1]

In [628]:
# get all the rows from result
sample = np.asarray(results)[sample_ids-1,:]

In [632]:
# subjects
sample_subjects = sample[:,4]
# title
sample_titles = sample[:,1]
# description
sample_descriptions = sample[:,2]
# title_descriptions
sample_title_descriptions = sample[:,3]

In [716]:
# all
sample_all = list(map(lambda x: " ".join(x), zip(sample_descriptions, sample_subjects)))


In [ ]:
# labelling for each set of documents using all methods
methods = ['count', 'count_weight', 'wmd']


In [661]:
model_s = np.array([label_docs(sample_subjects,vectorizer,method=x) for x in methods])


In [703]:
model_t = np.array([label_docs(sample_titles,vectorizer,method=x) for x in methods])
model_d = np.array([label_docs(sample_descriptions,vectorizer,method=x) for x in methods])
model_td = np.array([label_docs(sample_title_descriptions,vectorizer,method=x) for x in methods])

In [717]:
model_all = np.array([label_docs(sample_all,vectorizer,method=x) for x in methods])

In [688]:
# Pull predicted labels
pred_s = pull_labels(model_s)

In [704]:
pred_t = pull_labels(model_t)
pred_d = pull_labels(model_d)
pred_td = pull_labels(model_td)

In [718]:
pred_all = pull_labels(model_all)

In [728]:
# Calculate accuracy
accuracy_s = [get_accuracy(pred_s[i], truelabels) for i in range(3)]

In [705]:
accuracy_t = [get_accuracy(pred_t[i], truelabels) for i in range(3)]
accuracy_d = [get_accuracy(pred_d[i], truelabels) for i in range(3)]
accuracy_td = [get_accuracy(pred_td[i], truelabels) for i in range(3)]

In [719]:
accuracy_all = [get_accuracy(pred_all[i], truelabels) for i in range(3)]

In [797]:
print(accuracy_s)
print(accuracy_td)
print(accuracy_all)

[0.66666666666666663, 0.21428571428571427, 0.19047619047619047]
[0.19047619047619047, 0.5, 0.35714285714285715]
[0.2857142857142857, 0.26190476190476192, 0.26190476190476192]


In [723]:
pred_all[2] == truelabels

array([False,  True, False, False, False, False,  True,  True,  True,
       False,  True, False, False,  True, False,  True,  True,  True,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False], dtype=bool)

In [742]:
np.asarray([accuracy_s[0],accuracy_td[0],accuracy_all[0]])[np.newaxis].transpose()

array([[ 0.66666667],
       [ 0.19047619],
       [ 0.28571429]])

In [748]:
np.array(pred_s[0],pred_td[0],pred_all[0])

ValueError: only 2 non-keyword arguments accepted

In [765]:
# Add true labels to the results
table = np.column_stack((sample, truelabels))

In [766]:
# Put labels together. Only use subjects, title_descriptions
# order by method, then by type of documents
for i in range(3):
    table = np.column_stack((table,pred_s[i]))
    table = np.column_stack((table,pred_td[i]))
    table = np.column_stack((table,pred_all[i]))

In [790]:
table

array([['14001', 'Code Injection Attacks on HTML5-based Mobile Apps',
        '  HTML5-based mobile apps become more and more popular, mostly because they are\nmuch easier to be ported across different mobile platforms than native apps.\nHTML5-based apps are implemented using the standard web technologies, including\nHTML5, JavaScript and CSS; they depend on some middlewares, such as PhoneGap,\nto interact with the underlying OS.\n  Knowing that JavaScript is subject to code injection attacks, we have\nconducted a systematic study on HTML5-based mobile apps, trying to evaluate\nwhether it is safe to rely on the web technologies for mobile app development.\nOur discoveries are quite surprising. We found out that if HTML5-based mobile\napps become popular--it seems to go that direction based on the current\nprojection--many of the things that we normally do today may become dangerous,\nincluding reading from 2D barcodes, scanning Wi-Fi access points, playing MP4\nvideos, pairing with Blu

In [796]:
import csv
with open('sample_table.csv', 'w') as f:
    fieldnames = ['id','title','description','title_description','subjects','manual_label', 'by_subjects_count', 'by_title_desc_count', 'by_all_count', 'by_subjects_count_weight', 'by_title_desc_count_weight', 'by_all_count_weight', 'by_subjects_wmd', 'by_title_desc_wmd', 'by_all_wmd']
    writer = csv.writer(f)
    writer.writerow(fieldnames)
    writer.writerows(table)

In [ ]:
# Plot accuracy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
np.random.seed(1974)

# Generate Data
num = 20
x, y = np.random.random((2, num))
labels = np.random.choice(['a', 'b', 'c'], num)
df = pd.DataFrame(dict(x=x, y=y, label=labels))

groups = df.groupby('label')

# Plot
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=name)
ax.legend()

plt.show()

**Define all functions here**

In [512]:
# clean docs
def clean_doc(doc, stop_words):
    letters_only = re.sub("[^a-zA-Z]", " ", doc)
    letters_only = re.sub("  ", " ", letters_only)
    letters_list = letters_only.lower().split(" ")
    no_stopwords = [w for w in letters_list if w not in stopwords and w in model]
    return " ".join(no_stopwords)


In [385]:
import itertools

def word_dist(v1, v2):
    return norm(v1-v2)
def doc_dist(d1, d2):
    dist = 0
    for u in d1:
        dist += min(list(map(word_dist, np.repeat(u,len(d2),0), d2)))
    return dist

In [660]:
# write a function that does all the above jobs given any doc list
def label_docs(docs, tax_vectorizer, method='count'):
    """given a method (count, count_weight, wmd), label the docs using information in the taxonomy.
    docs: a list of documents to be labelled.
    tax_vectorizer: sk-learn vectorizer for the taxonomy
    method: can be count, count_weight, wmd.
    return the index of the label in collection_corpus and corresponding metrics"""
    docs = list((map(clean_doc, docs)))
    docs_vectorizer = CountVectorizer(min_df=1, stop_words=stopwords)
    docs_X = docs_vectorizer.fit_transform(docs).toarray()
    docs_vocab = np.asarray(docs_vectorizer.get_feature_names())
    tax_vocab = np.asarray(tax_vectorizer.get_feature_names())
    tax_X = X.toarray()
    if method == 'count':
        max_count = []
        for doc_distribution in docs_X:
            doc_words = list(docs_vocab[np.where(doc_distribution>0)])
            count = []
            for collection_distribution in tax_X:
                collection_words = list(tax_vocab[np.where(collection_distribution>0)])
                # print(len([x for x in sample_words if x in collection_words]))
                count.append(len([x for x in doc_words if x in collection_words]))
            max_count.append((max((val, idx) for (idx, val) in enumerate(count))))
        return max_count
    if method == 'count_weight':
        max_count = []
        for doc_distribution in docs_X:
            doc_words = list(docs_vocab[np.where(doc_distribution>0)])
            count = []
            for i, collection_distribution in enumerate(tax_X):
                collection_words = list(tax_vocab[np.where(collection_distribution>0)])
                # print(len([x for x in sample_words if x in collection_words]))
                cocurrence = [x for x in doc_words if x in collection_words]
                count.append(sum([collection_weights[i, idx] for idx, val in enumerate(tax_vocab) if val in cocurrence]))
            max_count.append((max((val, idx) for (idx, val) in enumerate(count))))
        return max_count
    if method == 'wmd':
        min_dist = []
        for i in range(len(docs)):
            doc_vector = np.asarray([np.asarray(model[x]) for x in np.asarray(docs_vectorizer.get_feature_names())[np.where(docs_X[i,:]>0)]])
            distances = [doc_dist(doc_vector, np.asarray(collection_vectors)[np.where(collection_weights[j]>0)]) for j in range(11)]
            val, idx = min((val, idx) for (idx, val) in enumerate(distances))
            min_dist.append((val,idx))
        return min_dist
    

In [687]:
# get labels from the results
def pull_labels(ob):
    return np.asarray(list(map(lambda o: np.asarray([all_labels[int(x[1])][1] for x in o]), ob)))

In [693]:
def get_accuracy(pred, test):
    """Percentage of correct labels"""
    return np.sum(pred==test)/len(test)

In [564]:
[x for x in collection_corpus]
labels = [()]

['science policy bioethics material transfer agreements open science research funding research integrity science technology workforce science education science policy economics technology regulations sanctity life justice science scientific beneficence scientific respect human dignity open source software open data open access institutional funding science military funding science philanthropic funding science research grants corporate funding science government funding science scientific misconduct publication ethics careers research citizen science science fairs',
 'engineering technology aerospace engineering architectural engineering automotive engineering bioengineering chemical engineering civil engineering control engineering digital imaging earthquake engineering electrical engineering electronics electronics engineering energy power environmental engineering equipment fire engineering fluidics forensic engineering geological engineering human factors engineering industrial eng

In [678]:
all_labels

[(0, 8912),
 (1, 3208),
 (2, 2948),
 (3, 7398),
 (4, 8600),
 (5, 9268),
 (6, 5930),
 (7, 2911),
 (8, 2092),
 (9, 7626),
 (10, 991)]

In [698]:
np.sum(pred_s[2]==truelabels)/len(truelabels)

0.19047619047619047

array([[2092, 7626, 7626, 7626, 7626, 7626, 7626,  991, 7626, 7626, 7626,
        2092,  991, 7626, 7626, 7626, 7626, 7626, 2092,  991, 2092, 2092,
        7626,  991, 2092,  991, 2092,  991, 2092, 2092,  991,  991, 2092,
        2092,  991,  991,  991, 2092, 2092,  991, 2092, 2092],
       [8912, 7626, 8912, 8600, 7626, 7626, 7626, 8912, 7626, 7626, 7626,
        8912, 8912, 8912, 2092, 2092, 7626, 7626, 8912, 2092, 8912, 8912,
        8912, 8912, 8912, 8912, 8912, 8912, 8912, 8912, 8912, 8912, 8912,
        8912, 8912, 8912, 8912, 8912, 8912, 8912, 8912, 8912],
       [8600, 7626, 5930, 7626, 2092, 7626, 2092, 5930, 7626, 2092, 5930,
        7626, 7626, 7626, 2092, 7626, 8600, 7626, 7626, 5930, 7626, 7626,
        7626, 7626, 7626, 7626, 7626, 8600, 8600, 7626, 8600, 7626, 7626,
        7626, 7626, 7626, 7626, 7626, 7626, 7626, 8600, 7626]])

In [ ]:
s = "A man, a plan, a canal: Panama"
def isPalindrome(s):
    s = s.lower()
    i = 0
    j = len(s) -1
    while i < j:
        criterion1 = s[i] >= 'a' and s[i] <= 'z' or s[i] >= 'A' and s[i] <= 'Z' or s[i] >= '0' and s[i] <= '9'
        criterion2 = s[j] >= 'a' and s[j] <= 'z' or s[j] >= 'A' and s[j] <= 'Z' or s[j] >= '0' and s[j] <= '9'
        if not criterion1:
            i += 1
        elif not criterion2:
            j += 1
        else:
            print("i:{},j:{}".format(s[i],s[j]))
            if s[i] != s[j]:
                return False
    return True
isPalindrome(s)